# Transportation problem


**Transportation problem is a special kind of Linear Programming Problem (LPP) in which goods are transported from a set of sources to a set of destinations subject to the supply and demand of the sources and destination respectively such that the total cost of transportation is minimized.**

We want to deliver  goods from two depots ( constantine and setif) to three destinations ( annaba, batna and bordj)

**the distances from each depot to each destination are given by the following list ( in hundred of km):**

('Constantine',  'Annaba') : 150<br>
('Constantine',  'batna')  : 120<br>
('Constantine',  'Borj')   : 250<br>
('Setif','Annaba') : 250<br>
('Setif','batna')  : 120<br>
('Setif','Borj')   : 65<br>

**the Capacity of depots are:**  
'Constantine':350<br>
'Setif':600   <br>
    
**the Demand at market j**  
'Annaba':325<br>
'batna':300<br>
'Borj':275   <br>
    
**Transportation Cost in dinars per case per hundreds of km**
2000 da/100km 

find the transportation planning to minimize the cost of transportation.
    
    


# Modelization 


Here is a general mathematical model  a transportation problem:

Let:
- $(i)$ be the index for sources, $(i = 1, 2, ..., m)$
- $(j)$ be the index for destinations, $(j = 1, 2, ...$(j)$, n)$
- $(dist_{ij})$ be the distance between the source $(i)$ and  destination $(j)$
- $(x_{ij})$ be the number of units to be transported from source $(i)$ to destination $(j)$
- $(c_{ij})$ be the cost of transporting one unit from source $(i)$ to destination $(j)$
- $(s_i)$ be the supply at source $(i)$
- $(d_j)$ be the demand at destination $(j)$


The linear programming model is:

Minimize: 
$$Z = \sum_{i=1}^{m} \sum_{j=1}^{n} c_{ij}x_{ij}$$
where $$c_{ij} =  f*dist_{ij}$$

Subject to:
- Supply constraints: $$\sum_{j=1}^{n} x_{ij} \leq s_i,  for~ all~ i  $$ 
- Demand constraints: $$\sum_{i=1}^{m} x_{ij} = d_j,   for ~ all ~ j $$
- Non-negativity constraints: $$ x_{ij} \geq 0 ,      for~ all~ (i,j) $$



# the resolution using the pyomo

In [3]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# Import
from pyomo.environ import *
 
# Creation of a Concrete Model
model = ConcreteModel()
 
## Define sets ##
#  Sets
#       i   product   / Constantine, Setif /
#       j   markets          / Annaba, batna, Borj / ;
model.i = Set(initialize=['Constantine','Setif'], doc='Product')
model.j = Set(initialize=['Annaba','batna', 'Borj'], doc='Markets')
 

model.a = Param(model.i, initialize={'Constantine':350,'Setif':600}, doc='Capacity of depot i in cases')
model.b = Param(model.j, initialize={'Annaba':325,'batna':300,'Borj':275}, doc='Demand at market j in cases')
 
dtab = {
    ('Constantine',  'Annaba') : 1.5,
    ('Constantine',  'batna')  : 1.2,
    ('Constantine',  'Borj')   : 2.5,
    ('Setif','Annaba') : 2.5,
    ('Setif','batna')  : 1.2,
    ('Setif','Borj')   : 0.65,
    }
model.d = Param(model.i, model.j, initialize=dtab, doc='Distance in hundreds of km')
#  Scalar f  freight in dinars per case per hundreds of km  /90/ ;
model.f = Param(initialize=2000, doc='Transportation Cost in dinars per case per hundreds of km')
 
#            c(i,j) = f * d(i,j) ;
def c_init(model, i, j):
  return model.f * model.d[i,j] 
  
model.c = Param(model.i, model.j, initialize=c_init, doc='Transport cost in dinars per case')
 
## Define variables ##

model.x = Var(model.i, model.j, bounds=(0.0,None), doc='Shipment quantities')
 
## Define constraints ##
# supply(i)   observe supply limit at plant i
# supply(i) .. sum (j, x(i,j)) =l= a(i)

def supply_rule(model, i):
  return sum(model.x[i,j] for j in model.j) <= model.a[i]

model.supply = Constraint(model.i, rule=supply_rule, doc='Observe supply limit at depot i')
# demand(j)   satisfy demand at market j ;  
# demand(j) .. sum(i, x(i,j)) =g= b(j);
def demand_rule(model, j):
  return sum(model.x[i,j] for i in model.i) >= model.b[j]
  
model.demand = Constraint(model.j, rule=demand_rule, doc='Satisfy demand at market j')
 
## Define Objective and solve ##

def objective_rule(model):
  return sum(model.c[i,j]*model.x[i,j] for i in model.i for j in model.j)

model.objective = Objective(rule=objective_rule, sense=minimize, doc='Define objective function')

opt = SolverFactory("glpk").solve(model)
model.x.display()

print('-----------------------------------')
print('----------solution------------------')
 
for i in model.i:
    for j in model.j:
        print(model.x[i,j], model.x[i,j].value)

x : Shipment quantities
    Size=6, Index=x_index
    Key                       : Lower : Value : Upper : Fixed : Stale : Domain
    ('Constantine', 'Annaba') :   0.0 : 325.0 :  None : False : False :  Reals
      ('Constantine', 'Borj') :   0.0 :   0.0 :  None : False : False :  Reals
     ('Constantine', 'batna') :   0.0 :  25.0 :  None : False : False :  Reals
          ('Setif', 'Annaba') :   0.0 :   0.0 :  None : False : False :  Reals
            ('Setif', 'Borj') :   0.0 : 275.0 :  None : False : False :  Reals
           ('Setif', 'batna') :   0.0 : 275.0 :  None : False : False :  Reals
-----------------------------------
----------solution------------------
x[Constantine,Annaba] 325.0
x[Constantine,batna] 25.0
x[Constantine,Borj] 0.0
x[Setif,Annaba] 0.0
x[Setif,batna] 275.0
x[Setif,Borj] 275.0


In [2]:
from pyomo.environ import *

# Create an abstract model
model = ConcreteModel()

# Sets
model.depots = Set(initialize=['Constantine', 'Setif'], doc='Depots')
model.destinations = Set(initialize=['Annaba', 'batna', 'Borj'], doc='Destinations')

# Parameters
# Distance in hundreds of km
dtab = {
    ('Constantine', 'Annaba'): 1.5,
    ('Constantine', 'batna'): 1.2,
    ('Constantine', 'Borj'): 2.5,
    ('Setif', 'Annaba'): 2.5,
    ('Setif', 'batna'): 1.2,
    ('Setif', 'Borj'): 0.65,
}

model.d = Param(model.depots, model.destinations, initialize=dtab, doc='Distance in hundreds of km')
model.capacity = Param(model.depots, initialize={'Constantine': 350, 'Setif': 600}, doc='Capacity of depot i in cases')
model.demand = Param(model.destinations, initialize={'Annaba': 325, 'batna': 300, 'Borj': 275}, doc='Demand at market j in cases')
model.freight_cost = Param(initialize=2000, doc='Freight in dinars per case per hundreds of km')

# Variables
model.x = Var(model.depots, model.destinations, within=NonNegativeReals, doc='Quantity of goods to be transported')

# Objective function
def objective_rule(model):
    return sum(model.x[i, j] * model.d[i, j] * model.freight_cost for i in model.depots for j in model.destinations)

model.obj = Objective(rule=objective_rule, sense=minimize)

# Constraints
def supply_constraint_rule(model, i):
    return sum(model.x[i, j] for j in model.destinations) <= model.capacity[i]

model.supply_constraints = Constraint(model.depots, rule=supply_constraint_rule)

def demand_constraint_rule(model, j):
    return sum(model.x[i, j] for i in model.depots) >= model.demand[j]

model.demand_constraints = Constraint(model.destinations, rule=demand_constraint_rule)

# Solve the model using a solver (e.g., GLPK)
solver = SolverFactory('glpk')
results = solver.solve(model)

# Print the results
model.pprint()
print("\nTransportation Cost (Optimal):", model.obj())
print("\nTransportation Plan:")
for i in model.depots:
    for j in model.destinations:
        print(f"From {i} to {j}: {model.x[i, j]()} cases")

# Export the results to a data file if needed
# model.results = results
# results.write()

4 Set Declarations
    d_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain              : Size : Members
        None :     2 : depots*destinations :    6 : {('Constantine', 'Annaba'), ('Constantine', 'batna'), ('Constantine', 'Borj'), ('Setif', 'Annaba'), ('Setif', 'batna'), ('Setif', 'Borj')}
    depots : Depots
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'Constantine', 'Setif'}
    destinations : Destinations
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'Annaba', 'batna', 'Borj'}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain              : Size : Members
        None :     2 : depots*destinations :    6 : {('Constantine', 'Annaba'), ('Constantine', 'batna'), ('Constantine', 'Borj'), ('Setif', 'Annaba'), ('Setif', 'batna'), ('Setif', 'Borj')}

4 Param Declara